In [225]:
with open("input.txt", 'r') as f:
    data = f.read().strip()
# data = "04005AC33890"
data = bin(int(data, 16))[2:].zfill(len(data)*4)
data

'011000100000110101111000000000001001100101100110000000001110010000110001100001000011000100101100110000000001101010001000100100010011111000011110000110000000001100010000111110100011001001000110010010011100110101011011100111011010011010111111110100010000011100000000001110100100111111011110100111000111000110000101100100110000000000111010010110010111100011000000000010100111000000000011110001000000000010100111000000000010010101000000000011010110000000100101100111010100000000001011001100000000001010001000000001111001001000100000000110111000100101000000000011100110000000010110100101001000000001001111000100100000000100010001100101000000000011000110000000001100000101000100000000001000000100000000001101000000000000010011010001000000000000100001001001111001101001011000000000011010111010010011110010101000010011000001000011001111001111010001000000001000011101010100000000010011011101001111011001111111011000010001100111001010010001100111011010011101100001100110010011100111011011111100100111100100110

In [226]:
z = "EE00D40C823060"
bin(int(z, 16))[2:].zfill(len(z)*4)

'11101110000000001101010000001100100000100011000001100000'

In [227]:
data[18:]

'111000000000001001100101100110000000001110010000110001100001000011000100101100110000000001101010001000100100010011111000011110000110000000001100010000111110100011001001000110010010011100110101011011100111011010011010111111110100010000011100000000001110100100111111011110100111000111000110000101100100110000000000111010010110010111100011000000000010100111000000000011110001000000000010100111000000000010010101000000000011010110000000100101100111010100000000001011001100000000001010001000000001111001001000100000000110111000100101000000000011100110000000010110100101001000000001001111000100100000000100010001100101000000000011000110000000001100000101000100000000001000000100000000001101000000000000010011010001000000000000100001001001111001101001011000000000011010111010010011110010101000010011000001000011001111001111010001000000001000011101010100000000010011011101001111011001111111011000010001100111001010010001100111011010011101100001100110010011100111011011111100100111100100110000000001010110010

In [228]:
def parse_literal(text):
    N = len(text)
    p = 0
    output = ""
    has_next = True
    while(has_next and p + 4 <= N):
        has_next = text[p] == '1'
        output += text[p+1:p+5]
        p += 5
    assert output, "literal too short to be parsed"
    return int(output, 2), p

In [229]:
parse_literal("101111111000101000")

(2021, 15)

In [230]:
def apply_operator(t, values):
    if t == 0:
        x = sum(values)
    elif t == 1:
        x = np.prod(values)
    elif t == 2:
        x = min(values)
    elif t == 3:
        x = max(values)
    elif t == 4:
        raise ValueError("literal package not handled")
    elif t == 5:
        assert len(values) == 2, "invalid length for greater than"
        x = 1 if (values[0] > values[1]) else 0
    elif t == 6:
        assert len(values) == 2, "invalid length for less than"
        x = 1 if values[0] < values[1] else 0
    elif t == 7:
        assert len(values) == 2, "invalid length for equal"
        x = 1 if values[0] == values[1] else 0
    else:
        raise ValueError("unknown package type")
    return x

In [231]:
def parse_packet(packet):
    print("packet", packet)
    pver, ptype = int(packet[:3],2), int(packet[3:6],2)
    print("ver", pver)
    print("type", ptype)
    p = 6
    versions = [pver]
    types = [ptype]
    values = []
    if ptype == 4:
        value, parsed = parse_literal(packet[p:])
        print(f"literal type: {value}")
        return versions, types, parsed + 6, value
    lentype = packet[p]
    p += 1
    if lentype == '0': #nbits
        nbits = int(packet[p:p+15], 2)
        p += 15
        print("nbits", nbits)
        while(nbits > 6):
            print(packet[p:])
            v, t, d, x = parse_packet(packet[p:])
            versions.extend(v)
            types.append(t)
            values.append(x)
            p += d
            nbits -= d
            print("nbits", nbits)
        remainder = packet[p:]
    else: #npackets
        npackets = int(packet[p:p+11], 2)
        p += 11
        print("npackets", npackets)
        for i in range(npackets):
            v, t, d, x = parse_packet(packet[p:])
            versions.extend(v)
            types.append(t)
            values.append(x)
            p += d
        remainder = packet[p:]   
    print("type: ", ptype)
    value = apply_operator(ptype, values)
    return versions, types, p, value

In [235]:
versions, types, p, x = parse_packet(data)

packet 011000100000110101111000000000001001100101100110000000001110010000110001100001000011000100101100110000000001101010001000100100010011111000011110000110000000001100010000111110100011001001000110010010011100110101011011100111011010011010111111110100010000011100000000001110100100111111011110100111000111000110000101100100110000000000111010010110010111100011000000000010100111000000000011110001000000000010100111000000000010010101000000000011010110000000100101100111010100000000001011001100000000001010001000000001111001001000100000000110111000100101000000000011100110000000010110100101001000000001001111000100100000000100010001100101000000000011000110000000001100000101000100000000001000000100000000001101000000000000010011010001000000000000100001001001111001101001011000000000011010111010010011110010101000010011000001000011001111001111010001000000001000011101010100000000010011011101001111011001111111011000010001100111001010010001100111011010011101100001100110010011100111011011111100100111100

packet 11110011111001001101001110111000011011000101000000000100101001110110110101100011001010000001100011100000000010111100010101011001111011000111011101111100011100111001010011111001011000010000000001101001001100101011110011011111011011111001001110111010000000000101010011100100011111110010000111101001100000011000011100000000100001100011011001001101000000111100010000101001110100100001100011100000011000000000101101001101001011100101100000000010000000000000000010000111110001100100100000101001000001001100000000000001000011001000100000010001000010100001111010011000000000100011111000000000100010001000000000111011000111101110001110001100000100100001000000000110101000011011010000011100100011100101001100101000111001100000000100001100111000101000110010111010010001001001000110011100011010000
ver 7
type 4
literal type: 244
packet 1101001110111000011011000101000000000100101001110110110101100011001010000001100011100000000010111100010101011001111011000111011101111100011100111001010011111001011000010

In [238]:
ans2 = x